<a href="https://colab.research.google.com/github/arthurziegler/pytorch-deep-learning-course/blob/main/Notebooks/PyTorch_Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.8.1

     |████████████████████████████████| 7.0 MB 5.3 MB/s 
     |████████████████████████████████| 776.8 MB 19 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.


In [51]:
import torch
import torch.nn as nn
import torchtext.data as ttd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime

In [135]:
# Let's make some fake data!
data = {
    "label": [0, 1, 1],
    "data": [
        "I like eggs and ham.",
        "Eggs I like!",
        "Ham and eggs or just ham?",
    ]
}

In [136]:
df = pd.DataFrame(data)

In [137]:
df.head()

,label,data
0,0,I like eggs and ham.
1,1,Eggs I like!
2,1,Ham and eggs or just ham?


In [138]:
#df.to_csv('thedata.csv', index=False)

In [139]:
#!head thedata.csv

In [140]:
def inputPreProcessing(input_data):
    processed_data = pd.DataFrame()

    # Add space before final ponctuation to prepare for split
    for index, row in input_data.iteritems():
        if re.match(r'\W', row[-1]):
            input_data.iloc[index] = row[:-1] + ' ' + row[-1]

    processed_data = input_data.str.lower()
    processed_data = processed_data.str.rsplit(expand=True)

    # Change padding 'None' from post to pre-padding
    processed_data = processed_data.to_numpy()
    for index, document in enumerate(processed_data):
        while(document[-1] == None):
            document = np.roll(document, 1)
        processed_data[index] = document

    # Create dictionary that will work as the vocabulary of the dataset
    dataset = list(processed_data.flatten())
    current_idx = 2 #leave 0 and 1 for padding and unknow words
    word2idx = {}
    word2idx[None] = 1
    word2idx['<Unknown>'] = 0
    for word in dataset:
        if word not in word2idx:
            word2idx[word] = current_idx
            current_idx += 1
    print(word2idx)
    print(processed_data)

    # Change tokens from word to integer
    for idx_doc, document in enumerate(processed_data):
        for idx_word, word in enumerate(document):
            processed_data[idx_doc, idx_word] = word2idx[word]
            #print(idx_doc, document, idx_word, word)
    
    print(processed_data)
    return processed_data


input_processed = inputPreProcessing(df['data'])
input_processed

{None: 1, '<Unknown>': 0, 'i': 2, 'like': 3, 'eggs': 4, 'and': 5, 'ham': 6, '.': 7, '!': 8, 'or': 9, 'just': 10, '?': 11}
[[None 'i' 'like' 'eggs' 'and' 'ham' '.']
 [None None None 'eggs' 'i' 'like' '!']
 ['ham' 'and' 'eggs' 'or' 'just' 'ham' '?']]
[[1 2 3 4 5 6 7]
 [1 1 1 4 2 3 8]
 [6 5 4 9 10 6 11]]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


array([[1, 2, 3, 4, 5, 6, 7],
       [1, 1, 1, 4, 2, 3, 8],
       [6, 5, 4, 9, 10, 6, 11]], dtype=object)

In [9]:
ex = dataset.examples[0]

In [10]:
type(ex)

torchtext.data.example.Example

In [11]:
ex.data

['i', 'like', 'eggs', 'and', 'ham', '.']

In [12]:
ex.label

'0'

In [ ]:
train_dataset, test_dataset = dataset.split(0.66) # default is 0.7

In [ ]:
for ex in train_dataset.examples:
  print(ex.data)

['ham', 'and', 'eggs', 'or', 'just', 'ham', '?']
['eggs', 'i', 'like', '!']


In [ ]:
TEXT.build_vocab(train_dataset,)

In [ ]:
vocab = TEXT.vocab
type(vocab)

torchtext.vocab.Vocab

In [ ]:
vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f948ea02690>>,
            {'!': 4,
             '<pad>': 1,
             '<unk>': 0,
             '?': 5,
             'and': 6,
             'eggs': 2,
             'ham': 3,
             'i': 7,
             'just': 8,
             'like': 9,
             'or': 10})

In [ ]:
vocab.itos

['<unk>', '<pad>', 'eggs', 'ham', '!', '?', 'and', 'i', 'just', 'like', 'or']

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
train_iter, test_iter = ttd.Iterator.splits(
        (train_dataset, test_dataset), sort_key=lambda x: len(x.data),
        batch_sizes=(2, 2), device=device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
for inputs, targets in train_iter:
  print("inputs:", inputs, "shape:", inputs.shape)
  print("targets:", targets, "shape:", targets.shape)
  break

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


inputs: tensor([[ 1,  1,  1,  2,  7,  9,  4],
        [ 3,  6,  2, 10,  8,  3,  5]], device='cuda:0') shape: torch.Size([2, 7])
targets: tensor([1, 1], device='cuda:0') shape: torch.Size([2])


In [ ]:
for inputs, targets in test_iter:
  print("inputs:", inputs, "shape:", inputs.shape)
  print("targets:", targets, "shape:", targets.shape)
  break

inputs: tensor([[7, 9, 2, 6, 3, 0]], device='cuda:0') shape: torch.Size([1, 6])
targets: tensor([0], device='cuda:0') shape: torch.Size([1])


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
# Exericise: Figure out which sequence of integers goes with which sentence.